In [1]:
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from scrapy.selector import Selector

import re, requests, csv
import lxml

# Regex
regex_url = r"(http(s?):\/\/(www)?[A-Za-z0-9-?\/\.]+)"
regex_hastags = r"(#[A-Za-z0-9]{2,})"

# File url
file_url = '../datasets/reuters_health.txt'
dest_url = './run_data/latimes_scrapt_v2.csv'

# Basic list
csv_header = ['tweetid','date','title']
frame_header = ['tweetid','date','title','what']

# Support scripts
%run -i scripts.py

In [2]:
# -------------------------------------------------
frame = pd.read_table(file_url, sep='|', header= None, names = csv_header, encoding="utf-8")
frame.head()

,tweetid,date,title
0,586282503981375488,Thu Apr 09 21:40:16 +0000 2015,Los Angeles closes 500 medical marijuana shops...
1,586278524748750848,Thu Apr 09 21:24:27 +0000 2015,U.S. cuts poultry export forecast as deadly bi...
2,586273441801699328,Thu Apr 09 21:04:15 +0000 2015,Fears over Roundup herbicide residues prompt p...
3,586254712523096068,Thu Apr 09 19:49:50 +0000 2015,Liberia watchdog says some Ebola funds unaccou...
4,586243976333787137,Thu Apr 09 19:07:10 +0000 2015,Diabetes devices may interfere with avalanche ...


In [3]:
# -------------------------------------------------
frame.shape

(4719, 3)

In [4]:
# -------------------------------------------------
frame.loc[:,'title'][0]

'Los Angeles closes 500 medical marijuana shops, but hundreds remain http://reut.rs/1CvkXmm'

In [5]:
# -------------------------------------------------
# Split Url & Text
frame['url'] = frame['title'].map(lambda row: get_url(regex_url, row))
frame['title'] = frame['title'].map(split_txt_form_url)

http://reut.rs/1CvkXmm
http://reut.rs/1abCAQY
http://reut.rs/1DpfmCU
http://reut.rs/1Dp1vfF
http://reut.rs/1DoTDLl
http://reut.rs/1DoRX4s
http://reut.rs/1CuvM85
http://reut.rs/1CufHPU
http://reut.rs/1Cu6UgG
http://reut.rs/1DolhYO
http://reut.rs/1Cu2dU6
http://reut.rs/1CtdtAf
http://reut.rs/1Ct7r2B
http://reut.rs/1CrMAwn
http://reut.rs/1CrDIH4
http://reut.rs/1CqEV1z
http://reut.rs/1DlT2K8
http://reut.rs/1CqyhIC
http://reut.rs/1DlA8Dq
http://reut.rs/1CpZ6g0
http://reut.rs/1DlfOCa
http://reut.rs/1CpVnPu
http://reut.rs/1CpKux1
http://reut.rs/1Dl7UZo
http://reut.rs/1DkWkxs
http://reut.rs/1CpqB9m
http://reut.rs/1Co8ror
http://reut.rs/1Cn9e9f
http://reut.rs/1Cl0YGG
http://reut.rs/1CkO1g2
http://reut.rs/1CkEiXh
http://reut.rs/1Dh1oTj
http://reut.rs/1DgTTMg
http://reut.rs/1CkhlDC
http://reut.rs/1DgKYuc
http://reut.rs/1CjK0Zs
http://reut.rs/1CjpTKK
http://reut.rs/1DfWwhh
http://reut.rs/1DfQYDH
http://reut.rs/1CiqkVF
http://reut.rs/1DfIImZ
http://reut.rs/1Dfs4UJ
http://reut.rs/1DfgxET
http://reut

http://reut.rs/1xzmpEM
http://reut.rs/1B4QbDN
http://reut.rs/1xyKaNj
http://reut.rs/1xyEXoz
http://reut.rs/1B45fBr
http://reut.rs/1xxFWW4
http://reut.rs/1xxrNsa
http://reut.rs/1B3qWBT
http://reut.rs/1B37gxN
http://reut.rs/1B2KLcq
http://reut.rs/1B2vzMh
http://HealthCare.gov
http://reut.rs/1B1RTpi
http://reut.rs/1xvwDGk
http://reut.rs/1CRy8De
http://reut.rs/1B1Kc2u
http://reut.rs/1B1Ju5w
http://reut.rs/1xvg0ui
http://reut.rs/1xuRucG
http://reut.rs/1xuNcSE
http://reut.rs/1B1cOJn
http://reut.rs/1B0jeIt
http://reut.rs/1B0cRoI
http://reut.rs/1xtnSfT
http://reut.rs/1xubcbI
http://reut.rs/1xt5tQ8
http://reut.rs/1xsZ7QM
http://reut.rs/1xsOsFR
http://reut.rs/1xsHiBo
http://reut.rs/1AZqN2e
http://reut.rs/1xsrkY7
http://reut.rs/1AXzfPp
http://reut.rs/1xqgvWm
http://reut.rs/1AWrIQY
http://reut.rs/1xmm9sx
http://reut.rs/1ATakMZ
http://reut.rs/1ASzf39
http://reut.rs/1ASuwyx
http://reut.rs/1AN7k4L
http://reut.rs/1xgwB4S
http://reut.rs/1AMSBa5
http://reut.rs/1AMiF50
http://reut.rs/1xecg0e
http://reut.

http://reut.rs/1D6iWhc
http://reut.rs/1oBlxin
http://reut.rs/1oBfZUT
http://reut.rs/1oAVcko
http://reut.rs/1D53L7U
http://reut.rs/1DGiAjB
http://reut.rs/1D4blQb
http://reut.rs/1D3POqN
http://reut.rs/1D3En2y
http://reut.rs/1oz7QQS
http://reut.rs/1D3annk
http://reut.rs/1D36PkX
http://reut.rs/1D2Zcem
http://reut.rs/1wrWMDZ
http://reut.rs/1D2J6Bm
http://t.co/r
http://reut.rs/1oyBIN6
http://reut.rs/1D2BcIm
http://reut.rs/1oyw1Pi
http://reut.rs/1oyahDi
http://reut.rs/1nACVmr
http://reut.rs/1D1Iqwb
http://reut.rs/1oxQ4gN
http://reut.rs/1D1AbQG
http://reut.rs/1oxLoHZ
http://reut.rs/1uKqu4E
http://reut.rs/1D1jK6Z
http://reut.rs/1wra8An
http://reut.rs/1oxnTi1
http://reut.rs/1oxkhgb
http://reut.rs/1owI6oe
http://reut.rs/1CZTxpn
http://reut.rs/1CZkg5y
http://reut.rs/1ovoHEd
http://reut.rs/1ovlepc
http://reut.rs/1CXLuJK
http://reut.rs/1ov8u1J
http://reut.rs/1CXuEdX
http://reut.rs/1ouYBkF
http://reut.rs/1ouUgOg
http://reut.rs/1ouQdSe
http://reut.rs/1ouMLqM
http://reut.rs/1sQ4lE4
http://reut.rs/1CWZ0

http://reut.rs/1sR6duZ
http://reut.rs/1sQI07Z
http://reut.rs/1oJxAYA
http://reut.rs/1oJaBNx
http://reut.rs/1sP6v5I
http://reut.rs/1oJ30OW
http://reut.rs/1sOPMPY
http://reut.rs/1sOhgFp
http://reut.rs/1oIbZzQ
http://reut.rs/1sO5ue6
http://reut.rs/1oIbYvR
http://reut.rs/1oI2As2
http://reut.rs/1oHTmvW
http://reut.rs/1oH4vgw
http://reut.rs/1oGJkeB
http://reut.rs/1oGARrK
http://reut.rs/1oGneIX
http://reut.rs/1oFPBHh
http://reut.rs/1oFFXV8
http://reut.rs/1oFeyCG
http://reut.rs/1sJOLs9
http://reut.rs/1oEQOP0
http://reut.rs/1oECUwy
http://reut.rs/1sI6pwA
http://reut.rs/1sI1s71
http://reut.rs/1oDuntJ
http://reut.rs/1oDqhlm
http://t.co/C0S
http://reut.rs/1oCTQ6l
http://reut.rs/1oCDZox
http://reut.rs/1oCxulC
http://reut.rs/1oCxult
http://reut.rs/1oCvA4x
http://reut.rs/1oC6B13
http://reut.rs/1sG6P6y
http://reut.rs/1oB61AL
http://reut.rs/1ozvmLr
http://reut.rs/1ozsSfV
http://reut.rs/1oz3fvZ
http://reut.rs/1oyv1bO
http://reut.rs/1szy16W
http://reut.rs/1syUH7w
http://reut.rs/1syOc4m
http://reut.rs/1ou

http://reut.rs/1CsnPDy
BioCryst Pharma wins federal contract to continue making Ebola drug http://reut.rs/1xSCxFi


http://reut.rs/1xSCxFi
Quintiles, Quest Diagnostics to form clinical trial labs joint venture http://reut.rs/1xSqSq8


http://reut.rs/1xSqSq8
China aims to double doctor numbers as cure for healthcare woes http://reut.rs/1xRuSHj


http://reut.rs/1xRuSHj
Ali Health pens tie-up for online health testing platform http://reut.rs/1Cr0Zfq


http://reut.rs/1Cr0Zfq
Arizona governor says he has signed abortion bill into law http://reut.rs/1xQGLx3


http://reut.rs/1xQGLx3
One in five teens have tried e-cigarettes, a British study finds http://reut.rs/1xQFc2r


http://reut.rs/1xQFc2r
Pesticides on fruits, veggies linked with poorer semen quality http://reut.rs/1xQyDwN


http://reut.rs/1xQyDwN
J&amp;J, Glaxo settle U.S. lawsuit over allergy ads http://reut.rs/1xQrBIj


http://reut.rs/1xQrBIj
U.S. cancer incidence, mortality largely stable or decreasing http://reut.rs/1CqimNu


http:/

...

New Jersey boy's death is 1st linked directly to enterovirus: CNN http://reut.rs/1vCcETG


http://reut.rs/1vCcETG
Nebraska hospital prepares for arrival of Ebola patient http://reut.rs/1vBAZsS


http://reut.rs/1vBAZsS
Ebola patient in Dallas 'fighting for his life': U.S. health official http://reut.rs/10BeBGJ


http://reut.rs/10BeBGJ
Ebola patient in Dallas turns critical, no new U.S. cases http://reut.rs/1vBqqpE


http://reut.rs/1vBqqpE
High risk Ebola could reach France and UK by end-October, scientists calculate http://reut.rs/1vB7CHd


http://reut.rs/1vB7CHd
Aid workers ask where was WHO in Ebola outbreak? http://reut.rs/1vAeIeY


http://reut.rs/1vAeIeY
Ebola ruled out for ill passenger escorted off plane in Newark http://reut.rs/10zHWBA


http://reut.rs/10zHWBA
Ebola patient in Dallas turns critical, no new U.S. cases http://reut.rs/1vz15gf


http://reut.rs/1vz15gf
Washington hospital rules out Ebola for patient back from Nigeria http://reut.rs/10zjvnK


http://reut.rs/10zjvnK
Ebo

http://reut.rs/1plOdFB
UN entrepreneur taps African oil for child health http://reut.rs/1o9qXLq


http://reut.rs/1o9qXLq
U.S. nutrition program for mothers, infants sees falling demand http://reut.rs/1mmkYaE


http://reut.rs/1mmkYaE
Killings in Guinea show mistrust in Africa Ebola fight: WHO http://reut.rs/1mmeAQA


http://reut.rs/1mmeAQA
Human error seen in measles vaccination deaths in Syria: WHO http://reut.rs/1o8HZJG


http://reut.rs/1o8HZJG
Angelina Jolie surgery sparks surge in female cancer tests: study http://reut.rs/1mlt8jA


http://reut.rs/1mlt8jA
India caps prices of 36 more drugs: government source http://reut.rs/1o8xNAV


http://reut.rs/1o8xNAV
Scientists see risk of mutant airborne Ebola as remote http://reut.rs/1mkULct


http://reut.rs/1mkULct
U.N. to deploy Ebola mission as death toll reaches 2,630 http://reut.rs/1o7jqN6


http://reut.rs/1o7jqN6
Cerner and Athenahealth say integrating with Apple's mobile health service http://reut.rs/1mjW5MK


http://reut.rs/1mjW5MK
U.S

http://reut.rs/1tZwmZ3
Ebola vaccine from GSK fast-tracked into clinical trials http://reut.rs/1tZjkuM


http://reut.rs/1tZjkuM
Ebola causing huge damage to West Africa economies: development bank http://reut.rs/1tZfdyB


http://reut.rs/1tZfdyB
GSK Ebola vaccine fast-tracked into U.S., UK and African trials http://reut.rs/1pKSTJV


http://reut.rs/1pKSTJV
Ebola toll tops 1,550, continues to accelerate: WHO http://reut.rs/1pKPUkH


http://reut.rs/1pKPUkH
Doctor dies of Ebola in Nigeria's oil hub Port Harcourt http://reut.rs/1pKJ8eO


http://reut.rs/1pKJ8eO
Repros says its testosterone drug found superior in study http://reut.rs/1tXDuoK


http://reut.rs/1tXDuoK
Fishery mislabeling could mean more mercury than buyers bargain for http://reut.rs/1pILAT9


http://reut.rs/1pILAT9
Self management may reduce blood pressure: study http://reut.rs/1tWT04d


http://reut.rs/1tWT04d
Scientists find mild cases of MERS among patients' families http://reut.rs/1tWFQV0


http://reut.rs/1tWFQV0
Florida firm



http://reut.rs/1oeOb6h
Russia bans beef, cattle imports from Romania: watchdog http://reut.rs/1se0GOI


http://reut.rs/1se0GOI
Tobacco-derived 'plantibodies' enter the fight against Ebola http://reut.rs/1odNAlr


http://reut.rs/1odNAlr
Second Ebola patient wheeled into Atlanta hospital for treatment http://reut.rs/1odmrPt


http://reut.rs/1odmrPt
Scientific review finds aspirin significantly cuts cancer rates http://reut.rs/1scb8Xd


http://reut.rs/1scb8Xd
Cytori suspends enrollment in cell therapy trials http://reut.rs/1sbJh9x


http://reut.rs/1sbJh9x
Paying to lose: cost effectiveness of weight loss programs http://reut.rs/1sbERzx


http://reut.rs/1sbERzx
Ebola mortality rate expected to rise as outbreak runs its deadly course http://reut.rs/1sbfwWb


http://reut.rs/1sbfwWb
About half of heart procedure patients make end-of-life plans http://reut.rs/1sbfwFP


http://reut.rs/1sbfwFP
Midlife high blood pressure tied to later-life mental decline http://reut.rs/1ocbGx1


http://reut.rs

Extreme fat photos may undermine message on overweight risk http://reut.rs/1tsqqvB


http://reut.rs/1tsqqvB
Mississippi baby thought cured of HIV no longer in remission http://reut.rs/1tsmWcq


http://reut.rs/1tsmWcq
Sarepta's shares plunge on muscle disorder drug data http://reut.rs/1sD2SQn


http://reut.rs/1sD2SQn
Mississippi baby seemingly cured of HIV now has detectable levels of the virus after two years of not taking antiretroviral therapy


Massachusetts curbs compounding pharmacies after 2012 meningitis outbreak http://reut.rs/1sCYP6F


http://reut.rs/1sCYP6F
Severe childhood obesity shows a decline in New York City http://reut.rs/1trSGOT


http://reut.rs/1trSGOT
Shorter sleep may speed brain aging http://reut.rs/1trNE52


http://reut.rs/1trNE52
WTC cleanup workers may renew health claims: U.S. appeals court http://reut.rs/1trEhCa


http://reut.rs/1trEhCa
Combining aerobic, resistance exercise may be best for diabetes http://reut.rs/1trjFdm


http://reut.rs/1trjFdm
EU Commissio

In [6]:
# -------------------------------------------------
frame.head()

,tweetid,date,title,url
0,586282503981375488,Thu Apr 09 21:40:16 +0000 2015,Los Angeles closes 500 medical marijuana shops...,http://reut.rs/1CvkXmm
1,586278524748750848,Thu Apr 09 21:24:27 +0000 2015,U.S. cuts poultry export forecast as deadly bi...,http://reut.rs/1abCAQY
2,586273441801699328,Thu Apr 09 21:04:15 +0000 2015,Fears over Roundup herbicide residues prompt p...,http://reut.rs/1DpfmCU
3,586254712523096068,Thu Apr 09 19:49:50 +0000 2015,Liberia watchdog says some Ebola funds unaccou...,http://reut.rs/1Dp1vfF
4,586243976333787137,Thu Apr 09 19:07:10 +0000 2015,Diabetes devices may interfere with avalanche ...,http://reut.rs/1DoTDLl


In [7]:
# -------------------------------------------------
frame.loc[:,'title'][0]

'Los Angeles closes 500 medical marijuana shops, but hundreds remain '

In [8]:
frame_content = pd.read_csv(dest_url, sep='|', encoding="utf-8")
### Writing content into text file
### ================================
file_names = frame['tweetid'].values

for key, name in enumerate(file_names):
    
    # Construct the file name
    located_file = './training/' + str(name) + '.txt'
    
    try:
    
        # Writing on file
        contents = str(frame_content['contents'].iloc[key])
        print(key, ' : ' ,type(contents))
        if len(contents) > 10:
            # Open file for writing
            f= open(located_file,"w+")
            
            # Start writing content to file.
            f.write(contents)
            
            # close file connection
            f.close()
        else:
            print("[Out] key: ", key) 
    except:
        pass

### ================================

0  :  <class 'str'>
1  :  <class 'str'>
2  :  <class 'str'>
3  :  <class 'str'>
4  :  <class 'str'>
5  :  <class 'str'>
6  :  <class 'str'>
7  :  <class 'str'>
[Out] key:  7
8  :  <class 'str'>
9  :  <class 'str'>
10  :  <class 'str'>
11  :  <class 'str'>
12  :  <class 'str'>
13  :  <class 'str'>
14  :  <class 'str'>
15  :  <class 'str'>
16  :  <class 'str'>
17  :  <class 'str'>
18  :  <class 'str'>
19  :  <class 'str'>
20  :  <class 'str'>
21  :  <class 'str'>
22  :  <class 'str'>
23  :  <class 'str'>
24  :  <class 'str'>
25  :  <class 'str'>
26  :  <class 'str'>
27  :  <class 'str'>
28  :  <class 'str'>
29  :  <class 'str'>
30  :  <class 'str'>
31  :  <class 'str'>
32  :  <class 'str'>
33  :  <class 'str'>
34  :  <class 'str'>
35  :  <class 'str'>
36  :  <class 'str'>
37  :  <class 'str'>
38  :  <class 'str'>
[Out] key:  38
39  :  <class 'str'>
[Out] key:  39
40  :  <class 'str'>
[Out] key:  40
41  :  <class 'str'>
[Out] key:  41
42  :  <class 'str'>
[Out] key:  42
43  :  <class 'str

384  :  <class 'str'>
385  :  <class 'str'>
386  :  <class 'str'>
387  :  <class 'str'>
388  :  <class 'str'>
389  :  <class 'str'>
390  :  <class 'str'>
391  :  <class 'str'>
392  :  <class 'str'>
393  :  <class 'str'>
394  :  <class 'str'>
395  :  <class 'str'>
396  :  <class 'str'>
397  :  <class 'str'>
398  :  <class 'str'>
399  :  <class 'str'>
400  :  <class 'str'>
401  :  <class 'str'>
402  :  <class 'str'>
403  :  <class 'str'>
404  :  <class 'str'>
405  :  <class 'str'>
406  :  <class 'str'>
407  :  <class 'str'>
408  :  <class 'str'>
409  :  <class 'str'>
410  :  <class 'str'>
411  :  <class 'str'>
412  :  <class 'str'>
413  :  <class 'str'>
414  :  <class 'str'>
415  :  <class 'str'>
416  :  <class 'str'>
417  :  <class 'str'>
418  :  <class 'str'>
419  :  <class 'str'>
420  :  <class 'str'>
421  :  <class 'str'>
422  :  <class 'str'>
423  :  <class 'str'>
424  :  <class 'str'>
425  :  <class 'str'>
426  :  <class 'str'>
427  :  <class 'str'>
428  :  <class 'str'>
429  :  <c

762  :  <class 'str'>
763  :  <class 'str'>
764  :  <class 'str'>
765  :  <class 'str'>
766  :  <class 'str'>
767  :  <class 'str'>
768  :  <class 'str'>
769  :  <class 'str'>
770  :  <class 'str'>
771  :  <class 'str'>
772  :  <class 'str'>
773  :  <class 'str'>
774  :  <class 'str'>
775  :  <class 'str'>
776  :  <class 'str'>
777  :  <class 'str'>
778  :  <class 'str'>
779  :  <class 'str'>
780  :  <class 'str'>
781  :  <class 'str'>
782  :  <class 'str'>
783  :  <class 'str'>
[Out] key:  783
784  :  <class 'str'>
785  :  <class 'str'>
786  :  <class 'str'>
787  :  <class 'str'>
788  :  <class 'str'>
789  :  <class 'str'>
790  :  <class 'str'>
791  :  <class 'str'>
792  :  <class 'str'>
793  :  <class 'str'>
794  :  <class 'str'>
795  :  <class 'str'>
796  :  <class 'str'>
797  :  <class 'str'>
798  :  <class 'str'>
799  :  <class 'str'>
800  :  <class 'str'>
801  :  <class 'str'>
802  :  <class 'str'>
[Out] key:  802
803  :  <class 'str'>
804  :  <class 'str'>
805  :  <class 'str'>


1223  :  <class 'str'>
1224  :  <class 'str'>
1225  :  <class 'str'>
1226  :  <class 'str'>
1227  :  <class 'str'>
1228  :  <class 'str'>
[Out] key:  1228
1229  :  <class 'str'>
1230  :  <class 'str'>
1231  :  <class 'str'>
[Out] key:  1231
1232  :  <class 'str'>
1233  :  <class 'str'>
1234  :  <class 'str'>
1235  :  <class 'str'>
1236  :  <class 'str'>
1237  :  <class 'str'>
1238  :  <class 'str'>
1239  :  <class 'str'>
1240  :  <class 'str'>
1241  :  <class 'str'>
1242  :  <class 'str'>
1243  :  <class 'str'>
1244  :  <class 'str'>
[Out] key:  1244
1245  :  <class 'str'>
1246  :  <class 'str'>
1247  :  <class 'str'>
1248  :  <class 'str'>
1249  :  <class 'str'>
1250  :  <class 'str'>
1251  :  <class 'str'>
1252  :  <class 'str'>
1253  :  <class 'str'>
1254  :  <class 'str'>
1255  :  <class 'str'>
1256  :  <class 'str'>
1257  :  <class 'str'>
1258  :  <class 'str'>
1259  :  <class 'str'>
1260  :  <class 'str'>
1261  :  <class 'str'>
1262  :  <class 'str'>
1263  :  <class 'str'>
1264  

1583  :  <class 'str'>
[Out] key:  1583
1584  :  <class 'str'>
[Out] key:  1584
1585  :  <class 'str'>
[Out] key:  1585
1586  :  <class 'str'>
[Out] key:  1586
1587  :  <class 'str'>
[Out] key:  1587
1588  :  <class 'str'>
[Out] key:  1588
1589  :  <class 'str'>
[Out] key:  1589
1590  :  <class 'str'>
[Out] key:  1590
1591  :  <class 'str'>
[Out] key:  1591
1592  :  <class 'str'>
[Out] key:  1592
1593  :  <class 'str'>
[Out] key:  1593
1594  :  <class 'str'>
[Out] key:  1594
1595  :  <class 'str'>
[Out] key:  1595
1596  :  <class 'str'>
[Out] key:  1596
1597  :  <class 'str'>
[Out] key:  1597
1598  :  <class 'str'>
[Out] key:  1598
1599  :  <class 'str'>
[Out] key:  1599
1600  :  <class 'str'>
[Out] key:  1600
1601  :  <class 'str'>
[Out] key:  1601
1602  :  <class 'str'>
[Out] key:  1602
1603  :  <class 'str'>
[Out] key:  1603
1604  :  <class 'str'>
[Out] key:  1604
1605  :  <class 'str'>
[Out] key:  1605
1606  :  <class 'str'>
[Out] key:  1606
1607  :  <class 'str'>
1608  :  <class '

...

2388  :  <class 'str'>
[Out] key:  2388
2389  :  <class 'str'>
[Out] key:  2389
2390  :  <class 'str'>
[Out] key:  2390
2391  :  <class 'str'>
[Out] key:  2391
2392  :  <class 'str'>
[Out] key:  2392
2393  :  <class 'str'>
[Out] key:  2393
2394  :  <class 'str'>
[Out] key:  2394
2395  :  <class 'str'>
[Out] key:  2395
2396  :  <class 'str'>
[Out] key:  2396
2397  :  <class 'str'>
[Out] key:  2397
2398  :  <class 'str'>
[Out] key:  2398
2399  :  <class 'str'>
[Out] key:  2399
2400  :  <class 'str'>
[Out] key:  2400
2401  :  <class 'str'>
[Out] key:  2401
2402  :  <class 'str'>
[Out] key:  2402
2403  :  <class 'str'>
[Out] key:  2403
2404  :  <class 'str'>
[Out] key:  2404
2405  :  <class 'str'>
[Out] key:  2405
2406  :  <class 'str'>
[Out] key:  2406
2407  :  <class 'str'>
[Out] key:  2407
2408  :  <class 'str'>
[Out] key:  2408
2409  :  <class 'str'>
[Out] key:  2409
2410  :  <class 'str'>
[Out] key:  2410
2411  :  <class 'str'>
[Out] key:  2411
2412  :  <class 'str'>
[Out] key:  2412


2788  :  <class 'str'>
[Out] key:  2788
2789  :  <class 'str'>
[Out] key:  2789
2790  :  <class 'str'>
[Out] key:  2790
2791  :  <class 'str'>
[Out] key:  2791
2792  :  <class 'str'>
[Out] key:  2792
2793  :  <class 'str'>
[Out] key:  2793
2794  :  <class 'str'>
[Out] key:  2794
2795  :  <class 'str'>
[Out] key:  2795
2796  :  <class 'str'>
[Out] key:  2796
2797  :  <class 'str'>
[Out] key:  2797
2798  :  <class 'str'>
[Out] key:  2798
2799  :  <class 'str'>
[Out] key:  2799
2800  :  <class 'str'>
[Out] key:  2800
2801  :  <class 'str'>
[Out] key:  2801
2802  :  <class 'str'>
[Out] key:  2802
2803  :  <class 'str'>
[Out] key:  2803
2804  :  <class 'str'>
[Out] key:  2804
2805  :  <class 'str'>
[Out] key:  2805
2806  :  <class 'str'>
[Out] key:  2806
2807  :  <class 'str'>
[Out] key:  2807
2808  :  <class 'str'>
[Out] key:  2808
2809  :  <class 'str'>
[Out] key:  2809
2810  :  <class 'str'>
[Out] key:  2810
2811  :  <class 'str'>
[Out] key:  2811
2812  :  <class 'str'>
[Out] key:  2812


[Out] key:  3187
3188  :  <class 'str'>
[Out] key:  3188
3189  :  <class 'str'>
[Out] key:  3189
3190  :  <class 'str'>
[Out] key:  3190
3191  :  <class 'str'>
[Out] key:  3191
3192  :  <class 'str'>
[Out] key:  3192
3193  :  <class 'str'>
[Out] key:  3193
3194  :  <class 'str'>
[Out] key:  3194
3195  :  <class 'str'>
[Out] key:  3195
3196  :  <class 'str'>
[Out] key:  3196
3197  :  <class 'str'>
[Out] key:  3197
3198  :  <class 'str'>
[Out] key:  3198
3199  :  <class 'str'>
[Out] key:  3199
3200  :  <class 'str'>
[Out] key:  3200
3201  :  <class 'str'>
[Out] key:  3201
3202  :  <class 'str'>
[Out] key:  3202
3203  :  <class 'str'>
[Out] key:  3203
3204  :  <class 'str'>
[Out] key:  3204
3205  :  <class 'str'>
[Out] key:  3205
3206  :  <class 'str'>
[Out] key:  3206
3207  :  <class 'str'>
[Out] key:  3207
3208  :  <class 'str'>
[Out] key:  3208
3209  :  <class 'str'>
[Out] key:  3209
3210  :  <class 'str'>
[Out] key:  3210
3211  :  <class 'str'>
[Out] key:  3211
3212  :  <class 'str'>


3422  :  <class 'str'>
[Out] key:  3422
3423  :  <class 'str'>
[Out] key:  3423
3424  :  <class 'str'>
[Out] key:  3424
3425  :  <class 'str'>
[Out] key:  3425
3426  :  <class 'str'>
[Out] key:  3426
3427  :  <class 'str'>
[Out] key:  3427
3428  :  <class 'str'>
[Out] key:  3428
3429  :  <class 'str'>
[Out] key:  3429
3430  :  <class 'str'>
[Out] key:  3430
3431  :  <class 'str'>
[Out] key:  3431
3432  :  <class 'str'>
[Out] key:  3432
3433  :  <class 'str'>
[Out] key:  3433
3434  :  <class 'str'>
[Out] key:  3434
3435  :  <class 'str'>
[Out] key:  3435
3436  :  <class 'str'>
[Out] key:  3436
3437  :  <class 'str'>
[Out] key:  3437
3438  :  <class 'str'>
[Out] key:  3438
3439  :  <class 'str'>
[Out] key:  3439
3440  :  <class 'str'>
[Out] key:  3440
3441  :  <class 'str'>
[Out] key:  3441
3442  :  <class 'str'>
[Out] key:  3442
3443  :  <class 'str'>
[Out] key:  3443
3444  :  <class 'str'>
[Out] key:  3444
3445  :  <class 'str'>
[Out] key:  3445
3446  :  <class 'str'>
[Out] key:  3446


[Out] key:  3687
3688  :  <class 'str'>
[Out] key:  3688
3689  :  <class 'str'>
[Out] key:  3689
3690  :  <class 'str'>
[Out] key:  3690
3691  :  <class 'str'>
[Out] key:  3691
3692  :  <class 'str'>
[Out] key:  3692
3693  :  <class 'str'>
[Out] key:  3693
3694  :  <class 'str'>
[Out] key:  3694
3695  :  <class 'str'>
[Out] key:  3695
3696  :  <class 'str'>
[Out] key:  3696
3697  :  <class 'str'>
[Out] key:  3697
3698  :  <class 'str'>
[Out] key:  3698
3699  :  <class 'str'>
[Out] key:  3699
3700  :  <class 'str'>
[Out] key:  3700
3701  :  <class 'str'>
[Out] key:  3701
3702  :  <class 'str'>
[Out] key:  3702
3703  :  <class 'str'>
[Out] key:  3703
3704  :  <class 'str'>
[Out] key:  3704
3705  :  <class 'str'>
[Out] key:  3705
3706  :  <class 'str'>
[Out] key:  3706
3707  :  <class 'str'>
[Out] key:  3707
3708  :  <class 'str'>
[Out] key:  3708
3709  :  <class 'str'>
[Out] key:  3709
3710  :  <class 'str'>
[Out] key:  3710
3711  :  <class 'str'>
[Out] key:  3711
3712  :  <class 'str'>


In [8]:
# -------------------------------------------------
# Xpath issues
list_scrapt = ['//div[@class="StandardArticleBody_body"]/p/text()',
               '',
               '//div[@class="BylineBar_byline"]/span/a/text()']

In [9]:
# -------------------------------------------------
# Scrapt form web
df_temp = pd.DataFrame()
frame_urls = frame['url'].values

In [ ]:
# -------------------------------------------------
# Run for this jupyter notebook
title_attempt = True
for key, url in enumerate(frame_urls):
    try:
        # Start the connection
        res = requests.get(url)        

        # Check return header
        if res.status_code == 200:
            print("Header:  ", 200)
            print(url)
            
            # Initialize XPath
            xpath_selector = Selector(text=res.text)
            
            list_content = dict()
            
            # Scrapt Text with XPath
            contents = xpath_selector.xpath('//div[@class="StandardArticleBody_body"]/p/text()').extract()
            contents = join_txt(contents)
            print(len(contents))
#             # Tags
#             tags = xpath_selector.xpath('').extract()     
#             print(len(tags))
            # Author
            author = xpath_selector.xpath('//div[@class="BylineBar_byline"]/span/a/text()').extract()
            if len(author):
                author = author[0]
            else:
                author = ''
            print(len(author))
            list_content.update({'contents': contents,'tags': '','author': author})
            
            if title_attempt:
                print("TItle: ", title_attempt)
                df_temp = pd.DataFrame(columns = list(list_content.keys()))
                frame = pd.concat([frame, df_temp])
                print(frame.columns)                
                title_attempt = False
                
            for k_, v_ in list_content.items():
                print(k_," : " ,len(v_))
                frame.loc[key,k_] = v_
            print("\n")

        else:
            print("['ERRORS'] {} , at {}".format(res.status_code, url))
    except:
        pass
    

Header:   200
http://reut.rs/1CvkXmm
2458
16
TItle:  True
Index(['author', 'contents', 'date', 'tags', 'title', 'tweetid', 'url'], dtype='object')
contents  :  2458
tags  :  0
author  :  16




scripts.py:38: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  # Find all possible hastags


Header:   200
http://reut.rs/1abCAQY
2080
0
contents  :  2080
tags  :  0
author  :  0


Header:   200
http://reut.rs/1DpfmCU
2660
12
contents  :  2660
tags  :  0
author  :  12


Header:   200
http://reut.rs/1Dp1vfF
1512
0
contents  :  1512
tags  :  0
author  :  0


Header:   200
http://reut.rs/1DoTDLl
3618
13
contents  :  3618
tags  :  0
author  :  13


Header:   200
http://reut.rs/1DoRX4s
1760
11
contents  :  1760
tags  :  0
author  :  11


Header:   200
http://reut.rs/1CuvM85
1886
13
contents  :  1886
tags  :  0
author  :  13


Header:   200
http://reut.rs/1CufHPU
2563
14
contents  :  2563
tags  :  0
author  :  14


Header:   200
http://reut.rs/1Cu6UgG
1211
0
contents  :  1211
tags  :  0
author  :  0


Header:   200
http://reut.rs/1DolhYO
4858
13
contents  :  4858
tags  :  0
author  :  13


Header:   200
http://reut.rs/1Cu2dU6
4445
13
contents  :  4445
tags  :  0
author  :  13


Header:   200
http://reut.rs/1CtdtAf
4265
0
contents  :  4265
tags  :  0
author  :  0


Header:   200
http

Header:   200
http://reut.rs/1xUnniV
3460
14
contents  :  3460
tags  :  0
author  :  14


Header:   200
http://reut.rs/1xUdO3y
1541
13
contents  :  1541
tags  :  0
author  :  13


Header:   200
http://reut.rs/1xTRNlk
3981
13
contents  :  3981
tags  :  0
author  :  13


Header:   200
http://reut.rs/1Ct3ii6
2048
13
contents  :  2048
tags  :  0
author  :  13


Header:   200
http://reut.rs/1xTnQlw
1630
0
contents  :  1630
tags  :  0
author  :  0


Header:   200
http://reut.rs/1Ct3heb
2539
0
contents  :  2539
tags  :  0
author  :  0


Header:   200
http://reut.rs/1xTnQBK
3994
13
contents  :  3994
tags  :  0
author  :  13


Header:   200
http://reut.rs/1CsVPj2
4132
13
contents  :  4132
tags  :  0
author  :  13


Header:   200
http://reut.rs/1CsSHDM
2301
11
contents  :  2301
tags  :  0
author  :  11


Header:   200
http://reut.rs/1xTaqWr
2048
13
contents  :  2048
tags  :  0
author  :  13


Header:   200
http://reut.rs/1xSRiIm
2387
11
contents  :  2387
tags  :  0
author  :  11


Header:   200


Header:   200
http://reut.rs/1HHZfPW
4198
17
contents  :  4198
tags  :  0
author  :  17


Header:   200
http://reut.rs/1HHsqTh
2268
18
contents  :  2268
tags  :  0
author  :  18


Header:   200
http://reut.rs/1HGNoln
2561
12
contents  :  2561
tags  :  0
author  :  12


Header:   200
http://reut.rs/1HGFdW3
3860
13
contents  :  3860
tags  :  0
author  :  13


Header:   200
http://reut.rs/1xzDxxX
4469
14
contents  :  4469
tags  :  0
author  :  14


Header:   200
http://reut.rs/1xzzOjX
2482
14
contents  :  2482
tags  :  0
author  :  14


Header:   200
http://reut.rs/1HG9GDQ
3991
16
contents  :  3991
tags  :  0
author  :  16


Header:   200
http://reut.rs/1xzpB79
1452
0
contents  :  1452
tags  :  0
author  :  0


Header:   200
http://reut.rs/1HFvI9P
504
0
contents  :  504
tags  :  0
author  :  0


Header:   200
http://reut.rs/1HFj8XZ
4492
13
contents  :  4492
tags  :  0
author  :  13


Header:   200
http://reut.rs/1HEnIFP
1321
0
contents  :  1321
tags  :  0
author  :  0


Header:   200
http

...

Header:   200
http://reut.rs/1tuApfT
4319
17
contents  :  4319
tags  :  0
author  :  17


Header:   200
http://reut.rs/1tuAq3o
3081
0
contents  :  3081
tags  :  0
author  :  0


Header:   200
http://reut.rs/1pikKRz
2205
13
contents  :  2205
tags  :  0
author  :  13


Header:   200
http://reut.rs/1pihQfo
543
0
contents  :  543
tags  :  0
author  :  0


Header:   200
http://reut.rs/1tursmG
1401
0
contents  :  1401
tags  :  0
author  :  0


Header:   200
http://reut.rs/1pi7IDq
1555
0
contents  :  1555
tags  :  0
author  :  0


Header:   200
http://reut.rs/1tujzh2
664
0
contents  :  664
tags  :  0
author  :  0


Header:   200
http://reut.rs/1tu9eSi
6609
11
contents  :  6609
tags  :  0
author  :  11


Header:   200
http://reut.rs/1phC2y9
2148
14
contents  :  2148
tags  :  0
author  :  14


Header:   200
http://reut.rs/1phtqaI
3073
16
contents  :  3073
tags  :  0
author  :  16


Header:   200
http://reut.rs/1ttg76t
4635
10
contents  :  4635
tags  :  0
author  :  10


Header:   200
http://reu

Header:   200
http://reut.rs/1t3f0tS
1870
11
contents  :  1870
tags  :  0
author  :  11


Header:   200
http://reut.rs/1oVkPdL
3863
13
contents  :  3863
tags  :  0
author  :  13


Header:   200
http://reut.rs/1t2OO2y
2396
0
contents  :  2396
tags  :  0
author  :  0


Header:   200
http://reut.rs/1oVcczU
1334
0
contents  :  1334
tags  :  0
author  :  0


Header:   200
http://reut.rs/1oV8B4R
3326
10
contents  :  3326
tags  :  0
author  :  10


Header:   200
http://reut.rs/1t2JhJb
317
0
contents  :  317
tags  :  0
author  :  0


Header:   200
http://reut.rs/1oV44PL
1188
0
contents  :  1188
tags  :  0
author  :  0


Header:   200
http://reut.rs/1oV1Hwn
6572
17
contents  :  6572
tags  :  0
author  :  17


Header:   200
http://reut.rs/1t2yWx8
6572
17
contents  :  6572
tags  :  0
author  :  17


Header:   200
http://reut.rs/1oUX3hL
1078
0
contents  :  1078
tags  :  0
author  :  0


Header:   200
http://reut.rs/1t2l0mP
2976
11
contents  :  2976
tags  :  0
author  :  11


Header:   200
http://r

Header:   200
http://reut.rs/1syOc4m
5564
13
contents  :  5564
tags  :  0
author  :  13


Header:   200
http://reut.rs/1ouoQ8E
4877
11
contents  :  4877
tags  :  0
author  :  11


Header:   200
http://reut.rs/1orKY3B
6571
12
contents  :  6571
tags  :  0
author  :  12


Header:   200
http://reut.rs/1orpmEg
1817
13
contents  :  1817
tags  :  0
author  :  13


Header:   200
http://reut.rs/1oqKWsJ
4108
13
contents  :  4108
tags  :  0
author  :  13


Header:   200
http://reut.rs/1stCQP7
4512
12
contents  :  4512
tags  :  0
author  :  12


Header:   200
http://reut.rs/1strOcE
3863
14
contents  :  3863
tags  :  0
author  :  14


Header:   200
http://reut.rs/1strNFR
3039
15
contents  :  3039
tags  :  0
author  :  15


Header:   200
http://reut.rs/1st44pc
5000
13
contents  :  5000
tags  :  0
author  :  13


Header:   200
http://reut.rs/1opOsnc
5973
12
contents  :  5973
tags  :  0
author  :  12


Header:   200
http://reut.rs/1opEfHj
2194
0
contents  :  2194
tags  :  0
author  :  0


Header:   20

Header:   200
http://reut.rs/1s592YQ
3015
16
contents  :  3015
tags  :  0
author  :  16


Header:   200
http://reut.rs/1o6eN9v
2801
13
contents  :  2801
tags  :  0
author  :  13


Header:   200
http://reut.rs/1o5UQQm
1357
0
contents  :  1357
tags  :  0
author  :  0


Header:   200
http://reut.rs/1s4p1GE
1511
0
contents  :  1511
tags  :  0
author  :  0


Header:   200
http://reut.rs/1s4jRud
6409
14
contents  :  6409
tags  :  0
author  :  14


Header:   200
http://reut.rs/1s3Vv3Q
3199
18
contents  :  3199
tags  :  0
author  :  18


Header:   200
http://reut.rs/1s2CCOV
4705
10
contents  :  4705
tags  :  0
author  :  10


Header:   200
http://reut.rs/1o2Ao2M
749
0
contents  :  749
tags  :  0
author  :  0


Header:   200
http://reut.rs/1o1boJl
3913
13
contents  :  3913
tags  :  0
author  :  13


Header:   200
http://reut.rs/1o0XDtV
2332
0
contents  :  2332
tags  :  0
author  :  0


Header:   200
http://reut.rs/1o0XDdx
2063
0
contents  :  2063
tags  :  0
author  :  0


Header:   200
http://r

Header:   200
http://reut.rs/1rBrMz5
2975
12
contents  :  2975
tags  :  0
author  :  12


Header:   200
http://reut.rs/1rBo08J
1449
0
contents  :  1449
tags  :  0
author  :  0


Header:   200
http://reut.rs/1nDzr0w
5998
15
contents  :  5998
tags  :  0
author  :  15


Header:   200
http://reut.rs/1rBi1Rm
3166
12
contents  :  3166
tags  :  0
author  :  12


Header:   200
http://reut.rs/1rBdYV7
2195
12
contents  :  2195
tags  :  0
author  :  12


Header:   200
http://reut.rs/1rB7Uw6
3583
15
contents  :  3583
tags  :  0
author  :  15


Header:   200
http://reut.rs/1rAW6K9
4153
9
contents  :  4153
tags  :  0
author  :  9


Header:   200
http://reut.rs/1nD4vxB
2424
0
contents  :  2424
tags  :  0
author  :  0


Header:   200
http://reut.rs/1rAEjCX
3828
16
contents  :  3828
tags  :  0
author  :  16


Header:   200
http://reut.rs/1nCOPKy
2968
16
contents  :  2968
tags  :  0
author  :  16




In [ ]:
# -------------------------------------------------
frame.head()

In [ ]:
# -------------------------------------------------
frame.shape

In [ ]:
frame.info()

In [ ]:
# frame.to_csv('./run_data/reuters_scrapt_vs.csv', sep='|', encoding='utf-8', index=False)
frame.to_csv('./run_data/latimes_scrapt_v2.csv', sep='|', encoding='utf-8',index=False)